In [1]:
import nbimporter

In [2]:
import Wall
from Wall import Layer, Wall, ResponseFactor

Importing Jupyter notebook from Wall.ipynb


# 複数の壁体情報を保持するクラス
- 壁体に関する入力情報を辞書型で受け取り、壁体関連情報を保持するクラスをインスタンス化します。
- 複数の壁体情報を配列で保持します。

### Constructor parameters
```
d = {
    'Common': {
        'Region'               : # 地域の区分 1 ~ 8 の整数
        'TimeInterval'         : # 計算時間間隔[s]
        'ResponseFactorCalculationHours': 応答係数を作成する時間数[h]
    },
    'Walls': [
        {
            'Name'             : # 壁体名称
            'Layers' : [
                {
                    'Name'     : # 部材名称　string値
                    'Thick'    : # 厚さ[m]
                    'Cond'     : # 熱伝導率[W/(m・K)]
                    'SpecH'    : # 容積比熱[kJ/(m3・K)]
                }
            ] # 層構成の情報
            'OutHeatTrans'     : # 室外側熱伝達率[W/(m2・K)]
            'OutEmissiv'       : # 室外側放射率[-]
            'OutSolarAbs'      : # 室外側日射吸収率[-]
            'InConHeatTrans'   : # 室内対流熱伝達率[W/(m2･K)
            'InRadHeatTrans'   : # 内放射熱伝達率[W/(m2･K)]
        }
    ]
}
```
### Class Definition

In [3]:
class WallMng:
    
    # 初期化
    def __init__( self, d ):
        # 登録壁体インスタンスを設定
        self.__objWall = []
        self.WalldataRead( d )
        
        # 壁体登録数
        self.__lngNWall = len( self.__objWall ) - 1
        
        # 壁体名称→登録番号変換
        self.__dicWallname = {}
        for lngI, x in enumerate(self.__objWall):
            self.__dicWallname[x.Wall().Name()] = lngI

    # 壁体構成データの読み込みと応答係数の作成
    def WalldataRead( self, d ):
        
        # 時間間隔(s)
        dblDTime = d['Common']['TimeInterval']

        # 応答係数を作成する時間数(h)
        dblNcalTime = d['Common']['ResponseFactorCalculationHours']
        
        # 登録壁体インスタンスの配列を作成
        for d_wall in d['Walls']:
            
            # 壁体構成部材の情報を保持するクラスをインスタンス化
            layers = []
            for d_layers in d_wall['Layers']:
                layer = Layer( d_layers['Name'], d_layers['Cond'], d_layers['SpecH'], d_layers['Thick'] )
                layers.append(layer)
            
            # 壁体情報を保持するクラスをインスタンス化
            wall = Wall( d_wall['Name'], d_wall['OutHeatTrans'], d_wall['OutEmissiv'], d_wall['OutSolarAbs'], d_wall['InConHeatTrans'], d_wall['InRadHeatTrans'], layers )
            
            # 応答係数を保持するクラスをインスタンス化し、配列に格納
            rf = ResponseFactor( 'wall' , dblDTime, dblNcalTime, wall )
            self.__objWall.append(rf)
    
    
    # 壁体名称から二等辺三角波励振の貫流応答係数を返す
    def RFT(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].RFT()
    
    # 壁体名称から二等辺三角波励振の吸熱応答係数を返す
    def RFT(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].RFA()
    
    # 壁体名称から公比を返す
    def Row(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].Row()
    
    # 壁体名称から指数項別貫流応答係数を返す
    def RFT1(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].RFT1()
    
    # 壁体名称から指数項別吸熱応答係数を返す
    def RFA1(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].RFA1()

    # 壁体名称から貫流応答係数の初項を返す
    def RFT0(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].RFT0()
    
    # 壁体名称から吸熱応答係数の初項を返す
    def RFA0(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].RFA0()

    # 壁体名称から根の数を返す
    def Nroot(self, strWallname):
        return len(self.__objWall[self.__dicWallname[strWallname]].Alp())

    # 室内側熱伝達率
    def hi(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].Wall().hi()

    # 室内側対流熱伝達率
    def hic(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].Wall().hic()

    # 室内側放射熱伝達率
    def hir(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].Wall().hir()
    
    # 室外側熱伝達率
    def ho(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].Wall().ho()
    
    # 室外側放射率
    def Eo(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].Wall().Eo()
    
    # 室外側日射吸収率
    def Solas(self, strWallname):
        return self.__objWall[self.__dicWallname[strWallname]].Wall().Solas()

## Example

In [4]:
d = {
    'Common': {
        'Region': 6,
        'TimeInterval': 3600,
        'ResponseFactorCalculationHours': 50
    },
    'Walls': [
        {
            'Name': 'Ceiling',
            'Layers': [
                {'Name': 'GW16K', 'Cond': 0.045, 'Thick': 0.0, 'SpecH': 13.0},
                {'Name': 'GPB', 'Cond': 0.22, 'Thick': 0.0095, 'SpecH': 830.0}
            ],
            'OutHeatTrans' : 25.0,
            'OutEmissiv' : 0.90,
            'OutSolarAbs' : 0.80,
            'InConHeatTrans' : 6.10,
            'InRadHeatTrans' : 5.00
        },
        {
            'Name': 'Wall_SW',
            'Layers': [
                {'Name': 'GW16K', 'Cond': 0.045, 'Thick': 0.0, 'SpecH': 13.0},
                {'Name': 'GPB', 'Cond': 0.22, 'Thick': 0.0095, 'SpecH': 830.0}
            ],
            'OutHeatTrans' : 25.0,
            'OutEmissiv' : 0.90,
            'OutSolarAbs' : 0.80,
            'InConHeatTrans' : 4.10,
            'InRadHeatTrans' : 5.00
        }
    ]
}

wallmng = WallMng( d )

### 二等辺三角波励振の応答係数の取得

In [5]:
print('二等辺三角波励振の貫流応答係数(12項まで)：', wallmng.RFT('Wall_SW')[:12] )
print('二等辺三角波励振の吸熱応答係数(12項まで)：', wallmng.RFT('Wall_SW')[:12] )

二等辺三角波励振の貫流応答係数(12項まで)： [22.296909777290789, 0.0082716866190094811, -1.3622230646715657e-05, 0.00018646549422714754, 0.00013369925842922768, 6.9879893268850787e-05, 2.5983123442825005e-05, -6.9233581549734961e-07, -1.576870144098192e-05, -2.3504394762507257e-05, -2.6723776700224879e-05, -2.7236269032143136e-05]
二等辺三角波励振の吸熱応答係数(12項まで)： [22.296909777290789, 0.0082716866190094811, -1.3622230646715657e-05, 0.00018646549422714754, 0.00013369925842922768, 6.9879893268850787e-05, 2.5983123442825005e-05, -6.9233581549734961e-07, -1.576870144098192e-05, -2.3504394762507257e-05, -2.6723776700224879e-05, -2.7236269032143136e-05]


### 公比の取得

In [6]:
wallmng.Row('Wall_SW')

[0.9928258579038134,
 0.9719606100332444,
 0.8937582091081409,
 0.6417733981379583,
 0.17334351801883058,
 0.0009850678853794972,
 1.3351199205276994e-12,
 1.2479464629129513e-47]

### 指数項別応答係数の取得

In [7]:
print('指数項別貫流応答係数：', wallmng.RFT1('Wall_SW') )
print('指数項別吸熱応答係数：', wallmng.RFA1('Wall_SW') )

指数項別貫流応答係数： [-1.430545379809063e-05, 0.00024372565703479105, -0.0022088262704731318, 0.013348446575400824, -0.042043450546070324, 0.15566967520745661, 0.0063455718148028167, -0.0018281332766963275]
指数項別吸熱応答係数： [-1.0028561710209951e-06, 1.7054122973132944e-05, -0.0001539341973333933, 0.00092211109304576866, -0.0028453279931310439, 0.0099589504434710215, 0.00032339374392940956, 5.0442262225605761e-05]


### 応答係数の初項の取得

In [8]:
print('貫流応答の初項：', wallmng.RFT0('Wall_SW') )
print('吸熱応答の初項：', wallmng.RFA0('Wall_SW') )

貫流応答の初項： 0.867348781276
吸熱応答の初項： 22.2969097773


### 根の数の取得

In [9]:
wallmng.Nroot('Wall_SW')

8

### 熱伝達率等の取得

In [10]:
print('室内側熱伝達率　　：', wallmng.hi('Wall_SW') )
print('室内側対流熱伝達率：', wallmng.hic('Wall_SW') )
print('室内側放射熱伝達率：', wallmng.hir('Wall_SW') )
print('室外側熱伝達率　　：', wallmng.ho('Wall_SW') )
print('室外側放射率　　　：', wallmng.Eo('Wall_SW') )
print('室外側日射吸収率　：', wallmng.Solas('Wall_SW') )

室内側熱伝達率　　： 9.1
室内側対流熱伝達率： 4.1
室内側放射熱伝達率： 5.0
室外側熱伝達率　　： 25.0
室外側放射率　　　： 0.9
室外側日射吸収率　： 0.8
